In [1]:
import pandas as pd 
import numpy as np 
import json 
import ast

In [ ]:
# Cargamos el archivo a normalizar 
data = pd.read_csv('./dataset/movies_dataset.csv')

In [ ]:
#Establecemos una lista de las columnas que vamos a elimnar porque no utilizaremos en nuestro analisis
columnas_eliminar = ['adult', 'homepage', 'imdb_id', 'original_title', 'poster_path', 'video', 'vote_count']
#Las eliminamos de nuestro dataframe 
data = data.drop(columnas_eliminar, axis = 1)

In [ ]:
#Realizamos una copia de respaldo de nuestro dataframe por posibles errores futuros 
data_copy = data.copy()

In [ ]:
#Creamos una funcion para extraer el nombre de los generos de cada diccionario
def extraer_nombre(obj): 
    if isinstance(obj, str) and '{' in obj:
        L=[]
        for i in ast.literal_eval(obj):
            L.append(i['name']);
        return L

#La funcion literal_eval convierte la columna de texto a lista de diccionarios 

In [ ]:
#Remplazamos la columna de genres con la nueva lista de los generos en nuestro dataframe
data['genres']=data['genres'].apply(extraer_nombre)

In [ ]:
#Definimos otra funcion para extraer los nombres de las franquicias 
def extraer_nombre2(obj): 
    
    if isinstance(obj, str) and '{' in obj:
        dic = ast.literal_eval(obj)
        return dic['name']

In [ ]:
data['belongs_to_collection'] = data['belongs_to_collection'].apply(extraer_nombre2)

In [ ]:
#Aplicamos la funcion al resto de las columnas con el mismo formato 
data['production_companies']=data['production_companies'].apply(extraer_nombre)
data['production_countries']=data['production_countries'].apply(extraer_nombre)
data['spoken_languages']=data['spoken_languages'].apply(extraer_nombre)

In [ ]:
valores_nulos = data['budget'].isnull().sum()
valores_vacios = data['budget'].eq('').sum()

#Mostramos los resultados
print(f"Valores nulos en '{'budget'}': {valores_nulos}")
print(f"Valores vacíos en '{'budget'}': {valores_vacios}")

In [ ]:
valores_nulos = data['revenue'].isnull().sum()
valores_vacios = data['revenue'].eq('').sum()

#Mostramos los resultados
print(f"Valores nulos en '{'revenue'}': {valores_nulos}")
print(f"Valores vacíos en '{'revenue'}': {valores_vacios}")

In [ ]:
#Reemplazamos los valores nulos por un valor especifico
data['revenue'] = data['revenue'].fillna(0)

In [ ]:
valores_nulos = data['release_date'].isnull().sum()
print(f"Valores nulos en '{'release_date'}': {valores_nulos}")

In [ ]:
data = data.dropna(subset = ['release_date'])

In [ ]:
data['release_date'] = pd.to_datetime(data['release_date'], errors='coerce')

In [ ]:
data['release_year'] = data['release_date'].dt.year 

In [ ]:
#Contamos cuantos valores NaT tenemos en la columna y los eliminamos
count_nats = data['release_date'].isna().sum()
count_nats

In [ ]:
data = data.dropna(subset=['release_date'])

In [ ]:
#Creamos columna return 
#Para esto primero debemos convertir la columna budget a numerica 
data['budget'] = pd.to_numeric(data['budget'])

In [ ]:
#Creamos la columna return 
data['return'] = np.where((data['budget'] == 0) | (data['revenue'] == 0), 0, data['revenue'] / data['budget'])

In [ ]:
#Creamos columna del mes de lanzamiento y dia de la semana de lanzamiento
data['release_month'] = data['release_date'].dt.month_name(locale = 'es_ES').str.lower()

In [ ]:
data['release_day'] = data['release_date'].dt.day_name(locale='es_ES').str.lower()

In [ ]:
#Proceso terminado lo extraemos como un csv
data.to_csv("C:/Users/marti/Desktop/PI - MLOps/dataset/movies_clean.csv", index=False)
